<a href="https://colab.research.google.com/github/felixlys/ValueAtRisk_implementation/blob/main/ValueAtRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Penerapan Value at Risk (VaR) untuk memilih investasi saham yang memiliki resiko terkecil.


Misalkan seorang investor ingin menginvestasikan uang sebesar 1 Miliar Rupiah untuk membeli saham/kepemilikan salah satu dari tiga saham perusahaan perbankan di Indonesia, sebagai berikut :

1. Bank Rakyat Indonesia (BBRI.JK)
2. Bank Central Asia (BBCA.JK)
3. Bank Negara Indonesia (BBNIP.JK)

Untuk membuat keputusan investasi yang tepat, investor perlu mengukur risiko yang terkait dengan masing-masing saham. Salah satu statistik yang sering digunakan untuk mengukur risiko ini adalah Value at Risk (VaR), dengan definisi sebagai berikut ⁉

**Definisi 1 :** Misalkan $L$ merupakan return acak dari suatu investasi yang memiliki distribusi probabilitas tertentu, maka statistik Value At Risk (VaR) merupakan nilai yang memenuhi Persamaan berikut :
\begin{equation}
P(-L>VaR)=1\%
\end{equation}
dengan $-L$ menyatakan besar nilai kerugian (dengan nilai kerugian akan bernilai positif dan keuntungan menjadi negatif).


**Catatan :**Perhatikan jika rata-rata dari $L$ adalah $\mu$ maka rata-rata dari $-L$ adalah $-\mu$.

Untuk mengimplementasikan Formula untuk menghitung VaR, pertama tama akan didapatkan data return dari masing-masing saham.

In [ ]:
import numpy as np
import yfinance as yf
daftar=['BBRI.JK','BBCA.JK','BBNI.JK']
data = yf.download(daftar,'2023-1-1','2024-1-1')['Adj Close']
#Menjadikan data harga saham menjadi array
BBCAP=np.array(data['BBCA.JK'])
BBNIP=np.array(data['BBNI.JK'])
BBRIP=np.array(data['BBRI.JK'])
#Susun suatu fungsi untuk mendapatkan return dari masing masing asset
def LnRet(harga):
    ret = np.zeros(len(harga) - 1)
    for i in range(1, len(harga)):
        ret[i-1] = np.log(harga[i] / harga[i-1])
    return ret
#Return dari investasi sebesar 1 miliar rupiah
RetBBCAP=1000000000*LnRet(BBCAP)
RetBBNIP=1000000000*LnRet(BBNIP)
RetBBRIP=1000000000*LnRet(BBRIP)

[*********************100%%**********************]  3 of 3 completed


Langkah selanjutnya adalah menguji kenormalan dari masing-masing Return Saham dengan uji Kolmogrov-Smirnov.

In [ ]:
#BBCA
import matplotlib.pyplot as plt
import scipy.stats as stats
#Uji Kolmogrov Smirnov (Kenormalan Data)
ks_stat, ks_pvalue = stats.kstest(RetBBCAP, 'norm', args=(np.mean(RetBBCAP),np.var(RetBBCAP)))
print(f"Statistik Kolmogorov-Smirnov: {ks_stat}")
print(f"Nilai p: {ks_pvalue}")
#plt.figure(figsize=(10,6))
#plt.hist(RetBBCAP, bins=50, density=True, alpha=0.6, color='g', label='Histogram Return BBCA')

Statistik Kolmogorov-Smirnov: 0.4999998921065576
Nilai p: 1.5199965173285187e-55


Hasil uji Kolmogrov Smirnov menunjukkan bahwa nilai $p< 0.05$ yang artinya Hipotesis Nol

$H_{0}:$ "Data Return BBCA tidak berbeda secara signifikan dari distribusi normal dengan parameter mean dan varians yang sesuai dengan data return BBCA"

**DITOLAK**. Dengan kata lain, data Return BBCA tidak terdistribusi normal.


Untuk itu, perlu dilakukan pembersihan data dari outlier yang terjadi dengan metode sebagai berikut :

In [ ]:
#Cleaning Data
#Kuartil 1 dan 3
Q1BBCA=np.percentile(RetBBCAP,25)
Q3BBCA=np.percentile(RetBBCAP,75)
#Jarak Interkuartil
IQRBBCA=Q3BBCA-Q1BBCA
#Menentukan Batas Atas dan Batas Bawah Outlier
bawahBBCA=Q1BBCA-1.5*IQRBBCA
atasBBCA=Q3BBCA+1.5*IQRBBCA
#Buang Data Outlier
outlierBBCA=RetBBCAP[(RetBBCAP < bawahBBCA) | (RetBBCAP > atasBBCA)]
RetBBCAP_1 =RetBBCAP[(RetBBCAP>= bawahBBCA) & (RetBBCAP<= atasBBCA)]

In [ ]:
print(RetBBCAP_1.shape)

(224,)


In [ ]:
#Cek Kenormalan dengan Uji Kolmogrov Smirnov
#BBCA
import matplotlib.pyplot as plt
import scipy.stats as stats
#Uji Kolmogrov Smirnov (Kenormalan Data)
ks_stat, ks_pvalue = stats.kstest(RetBBCAP_1, 'norm', args=(np.mean(RetBBCAP_1),np.var(RetBBCAP_1)))
print(f"Statistik Kolmogorov-Smirnov: {ks_stat}")
print(f"Nilai p: {ks_pvalue}")

Statistik Kolmogorov-Smirnov: 0.4999998824094689
Nilai p: 2.6198917998437853e-52


Adapun data yang di cleaning masih belum terdistribusi normal. Oleh karena itu, perlu dipertimbangkan distribusi lain selain Normal yakni t-Student dan distribusi Gamma. Dengan melakukan uji Kolmogrov-Smirnov akan diuji kesesuaian  Return BBCA dan dist t-student :

In [ ]:
#Uji Kolmogrov Smirnov untuk data Return BBCA dan distribusi t-Student
ks_stat, ks_pvalue = stats.kstest(RetBBCAP_1, 't', args=(223,))
print(f"Statistik Kolmogorov-Smirnov: {ks_stat}")
print(f"Nilai p: {ks_pvalue}")

Statistik Kolmogorov-Smirnov: 0.4375
Nilai p: 1.5706510524869764e-39


Masih belum sesuai dengan distribusi t-student.



Selanjutnya, digunakan alternatif lain yakni dengan mentransformasikan data Return BBCA menjadi data yang terdistribusi normal standar yakni $Z\sim N(0,1)$.

**Definisi 2** (Teorema Limit Pusat) Misalkan $X_{i}$ merupakan data ke-$i$ untuk $i=1,2,3,4,\cdots,n$ maka nilai dari $Z_{i}$ adalah sebagai berikut
\begin{equation}
Z_{i}=\frac{X_{i}-\mu}{\sigma}
\end{equation}
dengan $\mu$ adalah rata-rata data dan $\sigma$ adalah simpangan baku data.

In [ ]:
Z_RetBBCAP_1=(RetBBCAP_1-np.mean(RetBBCAP_1))/np.std(RetBBCAP_1)
print(Z_RetBBCAP_1)

[ 0.04173412 -1.35523837  0.74232425  2.11844389 -0.66958676  0.753055
 -1.74484735  1.47319334  2.50504755 -0.30697274  0.39044098 -0.30697274
 -1.01075216 -0.31122604  0.04173412  0.38325115 -0.6423229   0.37443147
  1.6910905  -0.28625745  1.02295175 -0.93948352  0.69680543  1.01744696
 -0.93397873 -2.26739937  0.37443147  0.37349314 -0.62272226 -0.29193431
  0.7080999  -0.62463167  1.37065786 -0.28906482 -1.96316864  0.3783082
 -1.99248847 -0.98890139  0.38629584  2.08792321  0.04173412 -1.66092118
  1.40584995  0.04173412 -0.30697274  1.08474547  0.38733701  1.41388666
 -1.61232806 -0.29193431  0.37065571 -0.94786985  0.70241649 -0.28814975
 -0.6208227   0.37349243  0.70241649  0.37065571  1.02295022  0.36697822
  1.01200209  0.36336941  1.31940127  0.99083542 -0.59013184 -1.23242283
  0.04173412 -1.57091471  1.01200209  0.04173412  0.04173412 -0.92853385
  0.04173412 -1.26472609 -0.28718748 -0.28814975 -0.95351962  1.03698785
  0.04173412  1.64103657 -1.23593304  0.36247967  1.31

In [ ]:
print(np.mean(RetBBCAP_1))

-359940.3666825982


In [ ]:
#Uji Kolmogrov Smirnov
ks_stat, ks_pvalue = stats.kstest(Z_RetBBCAP_1, 'norm', args=(0,1))
print(f"Statistik Kolmogorov-Smirnov: {ks_stat}")
print(f"Nilai p: {ks_pvalue}")

Statistik Kolmogorov-Smirnov: 0.0791446715609555
Nilai p: 0.11449598279572537


Terbukti bahwa benar Data Return BBCA yang ditransformasikan menjadi nilai Z-Score terdistribusi normal dengan mean 0 dan varians 1.


Selanjutnya, berdasarkan Definisi 1, Value At Risk dapat didefinisikan sebagai berikut
\begin{equation}
P(-L>VaR)=0.01
\\=P\left(\frac{-L-(-\mu)}{\sigma}>\frac{VaR+\mu}{\sigma} \right)
\\=P\left(Z>\frac{VaR+\mu}{\sigma}\right)
\\=1-P\left(Z\leq\frac{VaR+\mu}{\sigma}\right)
\end{equation}
sehingga
\begin{equation}
P\left(Z\leq\frac{VaR+\mu}{\sigma}\right)=0.99
\end{equation}

In [ ]:
k = stats.norm.ppf(0.99)

print(f"Nilai k: {k}")

Nilai k: 2.3263478740408408


Didapat formula nilai VaR
\begin{equation}
\frac{VaR+\mu}{\sigma}=2.3263478740408408\Rightarrow VaR=-\mu+2.3263478740408408\sigma.
\end{equation}

In [ ]:
#HitungVaR BBCA
VaRBBCA=-1*np.mean(RetBBCAP_1)+k*np.std(RetBBCAP_1)
print(VaRBBCA)

20423777.432799198


Adapun VaR pada BBCA adalah sebagai berikut ⁉
\begin{equation}
VaR_{BBCA}=20423781
\end{equation}

Selanjutnya, digunakan formula yang sama untuk mendapatkan Value At Risk dari BBNI dan BBRI

In [ ]:
VaRBBNIP=-1*np.mean(RetBBNIP)+k*np.std(RetBBNIP)
VaRBBRIP=-1*np.mean(RetBBRIP)+k*np.std(RetBBRIP)
print(f"Nilai VaR untuk BBNI={VaRBBNIP}")
print(f"Nilai VaR untuk BBRI={VaRBBRIP}")

Nilai VaR untuk BBNI=28863592.70404543
Nilai VaR untuk BBRI=29678711.149665225


Didapat nilai VaR untuk BBNI dan BBRI adalah sebagai berikut :
\begin{equation}
VaR_{BBNI}=28863593\\
VaR_{BBRI}=29678711.
\end{equation}

Oleh karena itu, keputusan investasi yang diharapkan memiliki resiko terkecil adalah pembelian saham BBCA sebesar 1 Miliar Rupiah.